# Brain Tissue Segmentation using MONAI Frameworks

# SwinUNETR Unet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install MONAI and other dependencies
!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

# Import necessary libraries
import os
import glob
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import nibabel as nib
import SimpleITK as sitk

import monai
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    RandSpatialCropd, RandFlipd, RandRotate90d, SpatialCropd,
    EnsureTyped, MapTransform, Transform
)
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.networks.nets import SwinUNETR  # Changed from AttentionUnet to SwinUNETR
from monai.utils import set_determinism

from google.colab import drive
drive.mount("/content/drive")

# Set determinism for reproducibility
set_determinism(seed=0)

# Define the data directory
data_dir = '/content/drive/MyDrive/Dataset_Final/'  # Update this path if different

# Function to load data dictionaries
def get_data_dicts(data_dir, set_name):
    data_pattern = os.path.join(data_dir, set_name, 'IBSR_*')
    subject_dirs = glob.glob(data_pattern)
    data_dicts = []

    for subject_dir in subject_dirs:
        subject_name = os.path.basename(subject_dir)
        img_file = os.path.join(subject_dir, '{}.nii.gz'.format(subject_name))
        seg_file = os.path.join(subject_dir, '{}_seg.nii.gz'.format(subject_name))
        data_dicts.append({'image': img_file, 'label': seg_file})

    # Debug: Print shapes of first few images
    if len(data_dicts) > 0:
        sample_image = nib.load(data_dicts[0]['image']).get_fdata()
        sample_label = nib.load(data_dicts[0]['label']).get_fdata()
        print(f"Sample image shape: {sample_image.shape}")
        print(f"Sample label shape: {sample_label.shape}")

    return data_dicts

# Get training and validation data dictionaries
train_files = get_data_dicts(data_dir, 'Training_Set')
val_files = get_data_dicts(data_dir, 'Validation_Set')

# Custom Transform: CenterSpatialCropd
class CenterSpatialCropd(Transform):
    def __init__(self, keys, roi_size):
        self.keys = keys
        self.roi_size = roi_size

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            img = d[key]
            spatial_shape = img.shape[1:]  # Assuming (C, D, H, W)
            center = [dim // 2 for dim in spatial_shape]
            # Apply SpatialCropd with roi_center
            crop = SpatialCropd(keys=[key], roi_size=self.roi_size, roi_center=center)
            d = crop(d)
        return d

# Define training transformations
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    RandSpatialCropd(keys=['image', 'label'], roi_size=[96, 96, 96], random_size=False),
    RandFlipd(keys=['image', 'label'], prob=0.5, spatial_axis=0),
    RandRotate90d(keys=['image', 'label'], prob=0.5, max_k=3),
    EnsureTyped(keys=['image', 'label']),
])

# Define validation transformations with deterministic spatial cropping
val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    CenterSpatialCropd(keys=['image', 'label'], roi_size=[96, 96, 96]),  # Deterministic center crop
    EnsureTyped(keys=['image', 'label']),
])

# Use CacheDataset for faster data loading
num_workers = 4  # Adjust based on your system's capabilities

# Create training and validation datasets and dataloaders
train_ds = CacheDataset(
    data=train_files,
    transform=train_transforms,
    cache_rate=1.0,  # Cache all data
    num_workers=num_workers
)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=num_workers)

val_ds = CacheDataset(
    data=val_files,
    transform=val_transforms,
    cache_rate=1.0,  # Cache all data
    num_workers=num_workers
)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=num_workers)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model (using MONAI's SwinUNETR)
try:
    model = SwinUNETR(
        in_channels=1,
        out_channels=4,               # Number of classes
        img_size=[96, 96, 96],        # Should match the ROI size used in transforms
        feature_size=48,              # Base number of features
        use_checkpoint=False,         # Disable checkpointing if unsupported
    ).to(device)
    print("SwinUNETR model instantiated successfully!")
except TypeError as e:
    print(f"Error during model instantiation: {e}")
    print("Please refer to MONAI v0.9.1 documentation for supported parameters.")
    raise e  # Re-raise the exception after informing the user

# Optionally, load pre-trained weights if available
# model.load_state_dict(torch.load('/path/to/pretrained/model.pth'))

# Define loss function and optimizer
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Define metric for evaluation without to_onehot_y and softmax
dice_metric = DiceMetric(
    include_background=True,
    reduction="mean",
    get_not_nans=False,
)

# Training loop
max_epochs = 2000
val_interval = 2  # Do validation every 2 epochs
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []

for epoch in range(max_epochs):
    print('-' * 10)
    print('Epoch {}/{}'.format(epoch + 1, max_epochs))
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data['image'].to(device), batch_data['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print('{}/{}, train_loss: {:.4f}'.format(step, len(train_loader), loss.item()))
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print('Epoch {} average loss: {:.4f}'.format(epoch + 1, epoch_loss))

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            metric_sum = 0.0
            metric_count = 0
            for val_data in val_loader:
                val_images = val_data['image'].to(device)
                val_labels = val_data['label'].to(device)
                val_outputs = model(val_images)

                # Apply softmax to model outputs
                val_outputs_softmax = F.softmax(val_outputs, dim=1)

                # One-hot encode labels
                val_labels_onehot = F.one_hot(val_labels.squeeze(1).long(), num_classes=4)  # Shape: (1, H, W, D, C)
                val_labels_onehot = val_labels_onehot.permute(0, 4, 1, 2, 3).float()       # Shape: (1, C, H, W, D)

                # Update DiceMetric
                dice_metric(y_pred=val_outputs_softmax, y=val_labels_onehot)

                metric_count += 1

            # Compute the mean dice score
            metric = dice_metric.aggregate().item()
            dice_metric.reset()
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), 'best_metric_model_SWIN.pth')
                print('Saved new best metric model')
            print('Current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}'.format(
                epoch + 1, metric, best_metric, best_metric_epoch))

print('Training completed, best_metric: {:.4f} at epoch: {}'.format(best_metric, best_metric_epoch))

# Load the best model for evaluation
model.load_state_dict(torch.load('best_metric_model_SWINUNETR.pth'))

# Evaluation functions
def compute_dice_coefficient(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    intersection = np.logical_and(in1, in2).sum()
    volumes = in1.sum() + in2.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2. * intersection / volumes

def compute_hausdorff_distance(in1, in2, label=1):
    in1 = (in1 == label).astype(np.uint8)
    in2 = (in2 == label).astype(np.uint8)
    in1_sitk = sitk.GetImageFromArray(in1)
    in2_sitk = sitk.GetImageFromArray(in2)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(in1_sitk, in2_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    vol1 = in1.sum()
    vol2 = in2.sum()
    if vol1 + vol2 == 0:
        return np.NaN
    else:
        return abs(vol1 - vol2) / (vol1 + vol2)

# Evaluate on validation data
model.eval()
dice_scores = {i: [] for i in range(4)}  # Assuming 4 classes
hausdorff_distances = {i: [] for i in range(4)}
volumetric_differences = {i: [] for i in range(4)}

with torch.no_grad():
    for val_data in val_loader:
        val_images = val_data['image'].to(device)
        val_labels = val_data['label'].cpu().numpy()
        outputs = model(val_images)
        outputs = torch.argmax(F.softmax(outputs, dim=1), dim=1).cpu().numpy()  # Shape: (1, H, W, D)

        for label in range(4):  # For each class
            dice = compute_dice_coefficient(outputs[0], val_labels[0][0], label=label)
            hd = compute_hausdorff_distance(outputs[0], val_labels[0][0], label=label)
            vd = compute_volumetric_difference(outputs[0], val_labels[0][0], label=label)
            if not np.isnan(dice):
                dice_scores[label].append(dice)
            if not np.isnan(hd):
                hausdorff_distances[label].append(hd)
            if not np.isnan(vd):
                volumetric_differences[label].append(vd)

# Compute average metrics per class
for label in range(4):
    avg_dice = np.mean(dice_scores[label]) if dice_scores[label] else np.NaN
    avg_hd = np.mean(hausdorff_distances[label]) if hausdorff_distances[label] else np.NaN
    avg_vd = np.mean(volumetric_differences[label]) if volumetric_differences[label] else np.NaN
    print(f"Class {label}:")
    print(f"\tDice Coefficient = {avg_dice:.4f}")
    print(f"\tHausdorff Distance = {avg_hd:.4f}")
    print(f"\tVolumetric Difference = {avg_vd:.4f}")


In [ ]:
# Install MONAI and other dependencies
!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

# Import necessary libraries
import os
import glob
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import nibabel as nib
import SimpleITK as sitk

import monai
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    SpatialCropd, EnsureTyped, Transform
)
from monai.networks.nets import SwinUNETR
from monai.utils import set_determinism
from monai.visualize import blend_images

from google.colab import drive
drive.mount("/content/drive")

# Set determinism for reproducibility
set_determinism(seed=0)

# Define the data directory
data_dir = '/content/drive/MyDrive/Dataset_Final/'  # Update this path if different

# Function to load data dictionaries
def get_data_dicts(data_dir, set_name):
    data_pattern = os.path.join(data_dir, set_name, 'IBSR_*')
    subject_dirs = glob.glob(data_pattern)
    data_dicts = []

    for subject_dir in subject_dirs:
        subject_name = os.path.basename(subject_dir)
        img_file = os.path.join(subject_dir, '{}.nii.gz'.format(subject_name))
        seg_file = os.path.join(subject_dir, '{}_seg.nii.gz'.format(subject_name))
        data_dicts.append({'image': img_file, 'label': seg_file})

    return data_dicts

# Get training and validation data dictionaries
train_files = get_data_dicts(data_dir, 'Training_Set')
val_files = get_data_dicts(data_dir, 'Validation_Set')

# Custom Transform: CenterSpatialCropd
class CenterSpatialCropd(Transform):
    def __init__(self, keys, roi_size):
        self.keys = keys
        self.roi_size = roi_size

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            img = d[key]
            spatial_shape = img.shape[1:]  # Assuming (C, D, H, W)
            center = [dim // 2 for dim in spatial_shape]
            # Apply SpatialCropd with roi_center
            crop = SpatialCropd(keys=[key], roi_size=self.roi_size, roi_center=center)
            d = crop(d)
        return d

# Define validation transformations with deterministic spatial cropping
val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    CenterSpatialCropd(keys=['image', 'label'], roi_size=[96, 96, 96]),
    EnsureTyped(keys=['image', 'label']),
])

# Use CacheDataset for faster data loading
num_workers = 4  # Adjust based on your system's capabilities

# Create validation dataset and dataloader
val_ds = CacheDataset(
    data=val_files,
    transform=val_transforms,
    cache_rate=1.0,
    num_workers=num_workers
)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=num_workers)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model (using MONAI's SwinUNETR)
model = SwinUNETR(
    in_channels=1,
    out_channels=4,
    img_size=[96, 96, 96],
    feature_size=48,
    use_checkpoint=False,
).to(device)
print("SwinUNETR model instantiated successfully!")

# Load the trained model weights
model_weights_path = '/content/drive/MyDrive/Dataset_Final/best_metric_model_SWIN.pth'  # Update this path
if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path, map_location=device))
    print(f"Loaded model weights from {model_weights_path}")
else:
    raise FileNotFoundError(f"Model weights not found at {model_weights_path}. Please check the path.")

# Define evaluation functions
def compute_dice_coefficient(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    intersection = np.logical_and(in1, in2).sum()
    volumes = in1.sum() + in2.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2. * intersection / volumes

def compute_hausdorff_distance(in1, in2, label=1):
    in1 = (in1 == label).astype(np.uint8)
    in2 = (in2 == label).astype(np.uint8)
    in1_sitk = sitk.GetImageFromArray(in1)
    in2_sitk = sitk.GetImageFromArray(in2)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(in1_sitk, in2_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    vol1 = in1.sum()
    vol2 = in2.sum()
    if vol1 + vol2 == 0:
        return np.NaN
    else:
        return abs(vol1 - vol2) / (vol1 + vol2)

# Function to resample image to match reference image
def resample_to_match(image, reference_image):
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_image)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampled_image = resampler.Execute(image)
    return resampled_image

# Directory to save the segmented outputs
output_dir = '/content/drive/MyDrive/Segmented_Outputs/'  # Update this path as needed
os.makedirs(output_dir, exist_ok=True)

# Evaluate on validation data
model.eval()
dice_scores = {i: [] for i in range(4)}
hausdorff_distances = {i: [] for i in range(4)}
volumetric_differences = {i: [] for i in range(4)}

with torch.no_grad():
    for idx, val_data in enumerate(val_loader):
        val_images = val_data['image'].to(device)
        val_labels = val_data['label'].cpu().numpy()
        outputs = model(val_images)
        outputs = torch.argmax(F.softmax(outputs, dim=1), dim=1).cpu().numpy()

        # Save the segmented output before resampling for visualization
        segmented_output_unresampled = outputs[0].astype(np.uint8)
        unresampled_output_file = os.path.join(output_dir, f"segmented_sample_{idx + 1}_cropped.nii.gz")
        # Create a SimpleITK image
        segmented_image_unresampled = sitk.GetImageFromArray(segmented_output_unresampled)
        # Copy the image information from the cropped image
        cropped_image_sitk = sitk.GetImageFromArray(val_data['image'][0].cpu().numpy()[0])
        segmented_image_unresampled.CopyInformation(cropped_image_sitk)
        # Save the image
        sitk.WriteImage(segmented_image_unresampled, unresampled_output_file)

        # Now proceed with resampling and saving the full-size segmentation
        segmented_image = sitk.GetImageFromArray(outputs[0].astype(np.uint8))
        original_image = sitk.ReadImage(val_data['image_meta_dict']['filename_or_obj'][0])

        # Resample segmented image to match the original image's size
        if segmented_image.GetSize() != original_image.GetSize():
            segmented_image = resample_to_match(segmented_image, original_image)

        # Copy metadata
        segmented_image.CopyInformation(original_image)
        output_file = os.path.join(output_dir, f"segmented_sample_{idx + 1}.nii.gz")
        sitk.WriteImage(segmented_image, output_file)
        print(f"Saved segmented output for sample {idx + 1} to {output_file}")

        # Calculate metrics
        for label in range(4):
            dice = compute_dice_coefficient(outputs[0], val_labels[0][0], label=label)
            hd = compute_hausdorff_distance(outputs[0], val_labels[0][0], label=label)
            vd = compute_volumetric_difference(outputs[0], val_labels[0][0], label=label)
            if not np.isnan(dice):
                dice_scores[label].append(dice)
            if not np.isnan(hd):
                hausdorff_distances[label].append(hd)
            if not np.isnan(vd):
                volumetric_differences[label].append(vd)
        print(f"Processed {idx + 1}/{len(val_loader)} validation samples.")

# Compute average metrics per class
print("\n--- Evaluation Metrics on Validation Set ---")
for label in range(4):
    avg_dice = np.mean(dice_scores[label]) if dice_scores[label] else np.NaN
    avg_hd = np.mean(hausdorff_distances[label]) if hausdorff_distances[label] else np.NaN
    avg_vd = np.mean(volumetric_differences[label]) if volumetric_differences[label] else np.NaN
    print(f"Class {label}:")
    print(f"\tDice Coefficient = {avg_dice:.4f}")
    print(f"\tHausdorff Distance = {avg_hd:.4f}")
    print(f"\tVolumetric Difference = {avg_vd:.4f}")

# Visualization using MONAI
def visualize_monai(sample_index, slice_index, val_ds, output_dir):
    data = val_ds[sample_index]
    image = data['image']  # Shape: (C, D, H, W)
    label = data['label']  # Shape: (C, D, H, W)
    segmented_output_path = os.path.join(output_dir, f"segmented_sample_{sample_index + 1}_cropped.nii.gz")
    segmented_output = nib.load(segmented_output_path).get_fdata()

    # Convert numpy arrays to torch tensors
    image_tensor = torch.from_numpy(image).float()
    label_tensor = torch.from_numpy(label).float()
    segmented_output_tensor = torch.from_numpy(segmented_output[np.newaxis, ...]).float()  # Add channel dimension

    # Normalize image for visualization
    image_tensor = (image_tensor - image_tensor.min()) / (image_tensor.max() - image_tensor.min())

    # Select the slice
    # Assuming slice_index corresponds to the D dimension
    image_slice = image_tensor[0, slice_index, :, :]  # Shape: (H, W)
    label_slice = label_tensor[0, slice_index, :, :]  # Shape: (H, W)
    segmented_slice = segmented_output_tensor[0, slice_index, :, :]  # Shape: (H, W)

    # Blend images
    blended_gt = blend_images(image_slice.unsqueeze(0), label_slice.unsqueeze(0), alpha=0.5)
    blended_pred = blend_images(image_slice.unsqueeze(0), segmented_slice.unsqueeze(0), alpha=0.5)

    # Convert blended images to (H, W, C)
    blended_gt_np = blended_gt.squeeze().permute(1, 2, 0).numpy()
    blended_pred_np = blended_pred.squeeze().permute(1, 2, 0).numpy()

    # Plot the images
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    axs[0].imshow(blended_gt_np)
    axs[0].set_title('Original with Ground Truth')
    axs[0].axis('off')

    axs[1].imshow(blended_pred_np)
    axs[1].set_title('Original with Predicted Segmentation')
    axs[1].axis('off')

    plt.show()

# Example usage:
# sample_index = 0  # Index of the sample in the validation dataset
# slice_index = 48  # Index of the slice to visualize (should be within the D dimension range)

# visualize_monai(sample_index, slice_index, val_ds, output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading dataset: 100%|██████████| 5/5 [00:00<00:00,  6.50it/s]


SwinUNETR model instantiated successfully!


<ipython-input-11-aec94bd09fc4>:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights_path, map_location=device))


Loaded model weights from /content/drive/MyDrive/Dataset_Final/best_metric_model_SWIN.pth
Saved segmented output for sample 1 to /content/drive/MyDrive/Segmented_Outputs/segmented_sample_1.nii.gz
Processed 1/5 validation samples.
Saved segmented output for sample 2 to /content/drive/MyDrive/Segmented_Outputs/segmented_sample_2.nii.gz
Processed 2/5 validation samples.
Saved segmented output for sample 3 to /content/drive/MyDrive/Segmented_Outputs/segmented_sample_3.nii.gz
Processed 3/5 validation samples.
Saved segmented output for sample 4 to /content/drive/MyDrive/Segmented_Outputs/segmented_sample_4.nii.gz
Processed 4/5 validation samples.
Saved segmented output for sample 5 to /content/drive/MyDrive/Segmented_Outputs/segmented_sample_5.nii.gz
Processed 5/5 validation samples.

--- Evaluation Metrics on Validation Set ---
Class 0:
	Dice Coefficient = 0.9490
	Hausdorff Distance = 19.4543
	Volumetric Difference = 0.0205
Class 1:
	Dice Coefficient = 0.9019
	Hausdorff Distance = 19.6174


# Ensemble

In [ ]:

!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

import os
import glob
import numpy as np
import torch
import torch.nn.functional as F
import nibabel as nib
import SimpleITK as sitk

import monai
from monai.data import CacheDataset, DataLoader, decollate_batch
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    RandSpatialCropd, RandFlipd, RandRotate90d, SpatialCropd,
    EnsureTyped, Transform, RandGaussianNoised, RandAdjustContrastd,
    RandGaussianSmoothd
)
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.networks.nets import SwinUNETR, UNet
from monai.utils import set_determinism

# If running in Colab, make sure you have your Drive mounted:
# from google.colab import drive
# drive.mount("/content/drive")

set_determinism(seed=0)

# ----------------------------
# 1) Data setup
# ----------------------------
data_dir = '/content/drive/MyDrive/Dataset_Final/'  # Update path to your dataset

def get_data_dicts(data_dir, set_name):
    data_pattern = os.path.join(data_dir, set_name, 'IBSR_*')
    subject_dirs = glob.glob(data_pattern)
    data_dicts = []
    for subject_dir in subject_dirs:
        subject_name = os.path.basename(subject_dir)
        img_file = os.path.join(subject_dir, f"{subject_name}.nii.gz")
        seg_file = os.path.join(subject_dir, f"{subject_name}_seg.nii.gz")
        data_dicts.append({'image': img_file, 'label': seg_file})

    # Debug: Print shapes of one sample
    if len(data_dicts) > 0:
        sample_image = nib.load(data_dicts[0]['image']).get_fdata()
        sample_label = nib.load(data_dicts[0]['label']).get_fdata()
        print(f"Sample image shape: {sample_image.shape}")
        print(f"Sample label shape: {sample_label.shape}")

    return data_dicts

train_files = get_data_dicts(data_dir, 'Training_Set')
val_files = get_data_dicts(data_dir, 'Validation_Set')

# ----------------------------
# 2) Custom transform for center cropping in validation
# ----------------------------
class CenterSpatialCropd(Transform):
    def __init__(self, keys, roi_size):
        self.keys = keys
        self.roi_size = roi_size

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            img = d[key]
            spatial_shape = img.shape[1:]  # shape: (C, D, H, W)
            center = [dim // 2 for dim in spatial_shape]
            crop = SpatialCropd(keys=[key], roi_size=self.roi_size, roi_center=center)
            d = crop(d)
        return d

# ----------------------------
# 3) Transforms (with additional augmentations)
# ----------------------------
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    # Patch-based training: random crop to 96x96x96
    RandSpatialCropd(keys=['image', 'label'], roi_size=[96, 96, 96], random_size=False),
    # Data augmentations
    RandFlipd(keys=['image', 'label'], prob=0.5, spatial_axis=0),
    RandRotate90d(keys=['image', 'label'], prob=0.5, max_k=3),
    RandGaussianNoised(keys=['image'], prob=0.2),
    RandGaussianSmoothd(keys=['image'], prob=0.2),
    RandAdjustContrastd(keys=['image'], prob=0.2),
    EnsureTyped(keys=['image', 'label']),
])

val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    CenterSpatialCropd(keys=['image', 'label'], roi_size=[96, 96, 96]),
    EnsureTyped(keys=['image', 'label']),
])

# ----------------------------
# 4) CacheDataset and DataLoader
# ----------------------------
num_workers = 4
train_ds = CacheDataset(data=train_files, transform=train_transforms,
                        cache_rate=1.0, num_workers=num_workers)
val_ds = CacheDataset(data=val_files, transform=val_transforms,
                      cache_rate=1.0, num_workers=num_workers)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=num_workers)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# 5) Define two models for the ensemble
# ----------------------------
model_swin = SwinUNETR(
    in_channels=1,
    out_channels=4,  # number of classes
    img_size=[96, 96, 96],
    feature_size=48,
    use_checkpoint=False,
).to(device)

model_unet = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=4,  # number of classes
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# ----------------------------
# 6) Common: Loss & single optimizer for both models
# ----------------------------
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
# Combine parameters from both models into one optimizer
optimizer = torch.optim.Adam(
    list(model_swin.parameters()) + list(model_unet.parameters()),
    lr=1e-4
)

dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)

# ----------------------------
# 7) Single training loop that trains both models as an ensemble
# ----------------------------
max_epochs = 4000  # Using your requested 2000 epochs
val_interval = 2
best_metric = -1.0
best_metric_epoch = -1

# Set a path in your Google Drive to store the best ensemble checkpoint
model_save_path = '/content/drive/MyDrive/best_metric_model_ensemble.pth'

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"Epoch {epoch + 1}/{max_epochs}")
    model_swin.train()
    model_unet.train()

    epoch_loss = 0
    step = 0

    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data["image"].to(device), batch_data["label"].to(device)

        # Forward pass for each model
        logits_swin = model_swin(inputs)      # (B, 4, D, H, W)
        logits_unet = model_unet(inputs)      # (B, 4, D, H, W)

        # Ensemble the logits by simple average (can adjust weighting if desired)
        logits_ensemble = 0.5 * logits_swin + 0.5 * logits_unet

        # Compute loss using the ensemble logits
        loss = loss_function(logits_ensemble, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        print(f"{step}/{len(train_loader)}, train_loss: {loss.item():.4f}")

    epoch_loss /= step
    print(f"Epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    # ----------------------------
    # Validation
    # ----------------------------
    if (epoch + 1) % val_interval == 0:
        model_swin.eval()
        model_unet.eval()
        dice_metric.reset()

        with torch.no_grad():
            for val_data in val_loader:
                val_images = val_data["image"].to(device)
                val_labels = val_data["label"].to(device)

                # Forward pass
                val_swin = model_swin(val_images)
                val_unet = model_unet(val_images)

                # Ensemble by averaging logits
                val_ensemble = 0.5 * val_swin + 0.5 * val_unet

                # Convert to softmax probabilities
                val_ensemble_soft = F.softmax(val_ensemble, dim=1)

                # Convert labels to one-hot
                val_labels_onehot = F.one_hot(val_labels.squeeze(1).long(), num_classes=4)
                val_labels_onehot = val_labels_onehot.permute(0, 4, 1, 2, 3).float()

                dice_metric(y_pred=val_ensemble_soft, y=val_labels_onehot)

            metric = dice_metric.aggregate().item()
            dice_metric.reset()

            print(f"Validation mean dice: {metric:.4f}")
            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                # Save both models in a single checkpoint to Google Drive
                torch.save(
                    {
                        "model_swin": model_swin.state_dict(),
                        "model_unet": model_unet.state_dict(),
                    },
                    model_save_path
                )
                print(f"Saved new best ensemble model to: {model_save_path}")

            print(f"Current epoch: {epoch + 1} current mean dice: {metric:.4f} "
                  f"best mean dice: {best_metric:.4f} at epoch {best_metric_epoch}")

print("TRAINING COMPLETE")
print(f"Best ensemble metric: {best_metric:.4f} at epoch {best_metric_epoch}")

# ----------------------------
# 8) Load best ensemble from Google Drive for evaluation
# ----------------------------
checkpoint = torch.load(model_save_path, map_location=device)
model_swin.load_state_dict(checkpoint["model_swin"])
model_unet.load_state_dict(checkpoint["model_unet"])

model_swin.eval()
model_unet.eval()

# ----------------------------
# 9) Metric computation helpers
# ----------------------------
def compute_dice_coefficient(pred, gt, label=1):
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    intersection = np.logical_and(pred_binary, gt_binary).sum()
    volumes = pred_binary.sum() + gt_binary.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2.0 * intersection / volumes

def compute_hausdorff_distance(pred, gt, label=1):
    pred_binary = (pred == label).astype(np.uint8)
    gt_binary = (gt == label).astype(np.uint8)
    pred_sitk = sitk.GetImageFromArray(pred_binary)
    gt_sitk = sitk.GetImageFromArray(gt_binary)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(pred_sitk, gt_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(pred, gt, label=1):
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    vol_pred = pred_binary.sum()
    vol_gt = gt_binary.sum()
    if (vol_pred + vol_gt) == 0:
        return np.NaN
    return abs(vol_pred - vol_gt) / (vol_pred + vol_gt)

# ----------------------------
# 10) Evaluate best ensemble on validation set
# ----------------------------
dice_scores = {cls: [] for cls in range(4)}
hausdorff_scores = {cls: [] for cls in range(4)}
vol_diff_scores = {cls: [] for cls in range(4)}

with torch.no_grad():
    for val_data in val_loader:
        val_images = val_data["image"].to(device)  # shape: (B,1,D,H,W)
        val_labels = val_data["label"].cpu().numpy()  # shape: (B,1,D,H,W)

        # Ensemble inference
        val_swin = model_swin(val_images)
        val_unet = model_unet(val_images)
        val_ensemble = 0.5 * val_swin + 0.5 * val_unet

        # Argmax of ensemble
        val_ensemble_argmax = torch.argmax(val_ensemble, dim=1)  # (B, D,H,W)

        pred_np = val_ensemble_argmax.cpu().numpy()  # shape: (B, D,H,W)

        for b in range(pred_np.shape[0]):
            gt = val_labels[b, 0]  # shape: (D,H,W)
            pred = pred_np[b]      # shape: (D,H,W)

            # Compute metrics for each class
            for cls in range(4):
                d = compute_dice_coefficient(pred, gt, label=cls)
                hd = compute_hausdorff_distance(pred, gt, label=cls)
                vd = compute_volumetric_difference(pred, gt, label=cls)

                if not np.isnan(d):
                    dice_scores[cls].append(d)
                if not np.isnan(hd):
                    hausdorff_scores[cls].append(hd)
                if not np.isnan(vd):
                    vol_diff_scores[cls].append(vd)

# ----------------------------
# 11) Print final average metrics (ensemble)
# ----------------------------
for cls in range(4):
    avg_dice = np.mean(dice_scores[cls]) if len(dice_scores[cls]) else np.NaN
    avg_hd = np.mean(hausdorff_scores[cls]) if len(hausdorff_scores[cls]) else np.NaN
    avg_vd = np.mean(vol_diff_scores[cls]) if len(vol_diff_scores[cls]) else np.NaN
    print(f"Class {cls}: Dice={avg_dice:.4f}, Hausdorff={avg_hd:.4f}, VolumetricDiff={avg_vd:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.7/990.7 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 20.3 MB/s eta 0:00:00
Sample image shape: (256, 128, 256, 1)
Sample label shape: (256, 128, 256, 1)
Sample image shape: (256, 128, 256, 1)
Sample label shape: (256, 128, 256, 1)


Loading dataset:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [0.8370536 1.5       0.8370536 0.       ] to [  0.8370536   1.5         0.8370536 213.4510119]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [1.  1.5 1.  0. ] to [  1.           1.5          1.         255.00196078]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
Loading dataset:  20%|██        | 2/10 [00:00<00:03,  2.18it/s]/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [0.9375 1.5    0.9375 0.    ] to [  0.9375       1.5          0.9375     239.06459149]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
Loading dataset: 100%|██████████| 5/5 [00:01<00:00,  3.90it/s]


----------
Epoch 1/4000


/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages

1/10, train_loss: 0.8454
2/10, train_loss: 0.8114
3/10, train_loss: 0.7876
4/10, train_loss: 0.7396
5/10, train_loss: 0.7237
6/10, train_loss: 0.7724
7/10, train_loss: 0.7548
8/10, train_loss: 0.7754
9/10, train_loss: 0.7047
10/10, train_loss: 0.8399
Epoch 1 average loss: 0.7755
----------
Epoch 2/4000
1/10, train_loss: 0.7187
2/10, train_loss: 0.7064
3/10, train_loss: 0.6410
4/10, train_loss: 0.7688
5/10, train_loss: 0.6179
6/10, train_loss: 0.6577
7/10, train_loss: 0.7335
8/10, train_loss: 0.7158
9/10, train_loss: 0.6028
10/10, train_loss: 0.5931
Epoch 2 average loss: 0.6756


/usr/local/lib/python3.10/dist-packages/monai/metrics/utils.py:219: UserWarning: y_pred should be a binarized tensor.
  warnings.warn(f"{name} should be a binarized tensor.")


Streaming output truncated to the last 5000 lines.
8/10, train_loss: 0.0710
9/10, train_loss: 0.0862
10/10, train_loss: 0.0541
Epoch 3593 average loss: 0.0843
----------
Epoch 3594/4000
1/10, train_loss: 0.0546
2/10, train_loss: 0.0631
3/10, train_loss: 0.0849
4/10, train_loss: 0.3058
5/10, train_loss: 0.1447
6/10, train_loss: 0.1683
7/10, train_loss: 0.0623
8/10, train_loss: 0.0372
9/10, train_loss: 0.0692
10/10, train_loss: 0.0640
Epoch 3594 average loss: 0.1054
Validation mean dice: 0.9281
Current epoch: 3594 current mean dice: 0.9281 best mean dice: 0.9332 at epoch 3536
----------
Epoch 3595/4000
1/10, train_loss: 0.0669
2/10, train_loss: 0.0709
3/10, train_loss: 0.1384
4/10, train_loss: 0.0625
5/10, train_loss: 0.0481
6/10, train_loss: 0.0957
7/10, train_loss: 0.0772
8/10, train_loss: 0.0487
9/10, train_loss: 0.0716
10/10, train_loss: 0.0513
Epoch 3595 average loss: 0.0731
----------
Epoch 3596/4000
1/10, train_loss: 0.0708
2/10, train_loss: 0.0573
3/10, train_loss: 0.0464
4/10, t

# Ensemble Evaluation

In [ ]:
# Install required libraries
!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

# Import required libraries
import os
import numpy as np
import torch
import torch.nn.functional as F
import nibabel as nib
import SimpleITK as sitk
from monai.data import DataLoader, CacheDataset
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, EnsureTyped
)
from monai.networks.nets import SwinUNETR, UNet
from monai.utils import set_determinism

# Set determinism
set_determinism(seed=0)

# ----------------------------
# 1) Data setup
# ----------------------------
data_dir = '/content/drive/MyDrive/Dataset_Final/'  # Update path to your dataset

def get_data_dicts(data_dir, set_name):
    import glob
    data_pattern = os.path.join(data_dir, set_name, 'IBSR_*')
    subject_dirs = glob.glob(data_pattern)
    data_dicts = []
    for subject_dir in subject_dirs:
        subject_name = os.path.basename(subject_dir)
        img_file = os.path.join(subject_dir, f"{subject_name}.nii.gz")
        seg_file = os.path.join(subject_dir, f"{subject_name}_seg.nii.gz")
        data_dicts.append({'image': img_file, 'label': seg_file, 'name': subject_name})

    return data_dicts

val_files = get_data_dicts(data_dir, 'Validation_Set')

val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    EnsureTyped(keys=['image', 'label']),
])

# ----------------------------
# 2) CacheDataset and DataLoader for validation
# ----------------------------
num_workers = 4
val_ds = CacheDataset(data=val_files, transform=val_transforms,
                      cache_rate=1.0, num_workers=num_workers)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=num_workers)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# 3) Define the ensemble models
# ----------------------------
model_swin = SwinUNETR(
    in_channels=1,
    out_channels=4,  # number of classes
    img_size=[96, 96, 96],
    feature_size=48,
    use_checkpoint=False,
).to(device)

model_unet = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=4,  # number of classes
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# ----------------------------
# 4) Load the saved ensemble checkpoint
# ----------------------------
model_save_path = '/content/drive/MyDrive/best_metric_model_ensemble.pth'
checkpoint = torch.load(model_save_path, map_location=device)

model_swin.load_state_dict(checkpoint["model_swin"])
model_unet.load_state_dict(checkpoint["model_unet"])

model_swin.eval()
model_unet.eval()

# ----------------------------
# 5) Metric computation helpers
# ----------------------------
def compute_dice_coefficient(pred, gt, label=1):
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    intersection = np.logical_and(pred_binary, gt_binary).sum()
    volumes = pred_binary.sum() + gt_binary.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2.0 * intersection / volumes

def compute_hausdorff_distance(pred, gt, label=1):
    pred_binary = (pred == label).astype(np.uint8)
    gt_binary = (gt == label).astype(np.uint8)
    pred_sitk = sitk.GetImageFromArray(pred_binary)
    gt_sitk = sitk.GetImageFromArray(gt_binary)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(pred_sitk, gt_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(pred, gt, label=1):
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    vol_pred = pred_binary.sum()
    vol_gt = gt_binary.sum()
    if (vol_pred + vol_gt) == 0:
        return np.NaN
    return abs(vol_pred - vol_gt) / (vol_pred + vol_gt)

# ----------------------------
# 6) Evaluate the ensemble on the validation set and save segmented images
# ----------------------------
segmented_dir = '/content/drive/MyDrive/Dataset_Final/segmented_images/validation/'
os.makedirs(segmented_dir, exist_ok=True)

# Define the voxel spacing and origin
target_spacing = [1, 1.5, 1]  # Spacing: x, y, z
target_origin = [255, 0, 0]  # Origin: x, y, z

# Metric storage
overall_dice_scores = []
overall_hd_scores = []
overall_vd_scores = []

with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        val_images = val_data["image"].to(device)  # shape: (B,1,D,H,W)
        val_labels = val_data["label"].cpu().numpy()  # shape: (B,1,D,H,W)
        val_name = val_files[i]['name']  # Extract file name from the data dict

        # Ensemble inference
        val_swin = model_swin(val_images)
        val_unet = model_unet(val_images)
        val_ensemble = 0.5 * val_swin + 0.5 * val_unet

        # Argmax of ensemble
        val_ensemble_argmax = torch.argmax(val_ensemble, dim=1)  # (B, D,H,W)

        pred_np = val_ensemble_argmax.cpu().numpy()  # shape: (B, D,H,W)

        for b in range(pred_np.shape[0]):
            gt = val_labels[b, 0]  # shape: (D,H,W)
            pred = pred_np[b]      # shape: (D,H,W)

            # Resize the prediction to the original image size (256x128x256)
            original_shape = gt.shape  # Assuming ground truth is in original size
            pred_resized = torch.nn.functional.interpolate(
                torch.tensor(pred).unsqueeze(0).unsqueeze(0).float(),
                size=original_shape,
                mode="nearest"
            ).squeeze().numpy()

            # Create affine matrix for desired spacing and origin
            affine = np.array([
                [target_spacing[0], 0, 0, target_origin[0]],
                [0, target_spacing[1], 0, target_origin[1]],
                [0, 0, target_spacing[2], target_origin[2]],
                [0, 0, 0, 1],
            ])

            # Save the resized predicted segmentation as a NIfTI file
            output_path = os.path.join(segmented_dir, f"Segmented_{val_name}.nii.gz")
            pred_img = nib.Nifti1Image(pred_resized.astype(np.int16), affine)
            nib.save(pred_img, output_path)
            print(f"Saved segmentation to {output_path}")

            # Metrics for CSF (1), GM (2), WM (3)
            labels = [0,1, 2, 3]
            dice_scores = []
            hd_scores = []
            vd_scores = []

            for label in labels:
                dice = compute_dice_coefficient(pred_resized, gt, label=label)
                hd = compute_hausdorff_distance(pred_resized, gt, label=label)
                vd = compute_volumetric_difference(pred_resized, gt, label=label)

                if not np.isnan(dice):
                    dice_scores.append(dice)
                if not np.isnan(hd):
                    hd_scores.append(hd)
                if not np.isnan(vd):
                    vd_scores.append(vd)

            # Append per-image metrics to overall metrics
            overall_dice_scores.append(np.mean(dice_scores))
            overall_hd_scores.append(np.mean(hd_scores))
            overall_vd_scores.append(np.mean(vd_scores))

            print(f"Image {val_name} Metrics:")
            print(f"  Mean Dice: {np.mean(dice_scores):.4f}")
            print(f"  Mean Hausdorff Distance: {np.mean(hd_scores):.4f}")
            print(f"  Mean Volumetric Difference: {np.mean(vd_scores):.4f}")

# ----------------------------
# 7) Print final mean metrics
# ----------------------------
print("\nFinal Mean Metrics Across All Images:")
print(f"Mean Dice: {np.mean(overall_dice_scores):.4f}")
print(f"Mean Hausdorff Distance: {np.mean(overall_hd_scores):.4f}")
print(f"Mean Volumetric Difference: {np.mean(overall_vd_scores):.4f}")


Loading dataset: 100%|██████████| 5/5 [00:00<00:00,  6.48it/s]
<ipython-input-9-4c99e20ccd07>:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model

Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_13.nii.gz
Image IBSR_13 Metrics:
  Mean Dice: 0.9320
  Mean Hausdorff Distance: 12.4971
  Mean Volumetric Difference: 0.0266
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_14.nii.gz
Image IBSR_14 Metrics:
  Mean Dice: 0.9543
  Mean Hausdorff Distance: 10.5668
  Mean Volumetric Difference: 0.0104
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_11.nii.gz
Image IBSR_11 Metrics:
  Mean Dice: 0.9424
  Mean Hausdorff Distance: 9.2214
  Mean Volumetric Difference: 0.0106
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_17.nii.gz
Image IBSR_17 Metrics:
  Mean Dice: 0.9480
  Mean Hausdorff Distance: 12.2100
  Mean Volumetric Difference: 0.0120
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmen

# Checking Proper Segmentation

In [ ]:
!pip install SimpleITK
import os
import numpy as np
import nibabel as nib
import SimpleITK as sitk

#############################
# 1) Define metric functions
#############################
def compute_dice_coefficient(pred, gt, label=1):
    """
    Compute the Dice coefficient for a single label.
    """
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    intersection = np.logical_and(pred_binary, gt_binary).sum()
    volumes = pred_binary.sum() + gt_binary.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2.0 * intersection / volumes

def compute_hausdorff_distance(pred, gt, label=1):
    """
    Compute the Hausdorff distance for a single label.
    Uses SimpleITK's HausdorffDistanceImageFilter.
    """
    pred_binary = (pred == label).astype(np.uint8)
    gt_binary = (gt == label).astype(np.uint8)
    pred_sitk = sitk.GetImageFromArray(pred_binary)
    gt_sitk = sitk.GetImageFromArray(gt_binary)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(pred_sitk, gt_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(pred, gt, label=1):
    """
    Compute the Volumetric Difference for a single label:
    abs(vol_pred - vol_gt) / (vol_pred + vol_gt).
    """
    pred_binary = (pred == label)
    gt_binary = (gt == label)
    vol_pred = pred_binary.sum()
    vol_gt = gt_binary.sum()
    if (vol_pred + vol_gt) == 0:
        return np.NaN
    return abs(vol_pred - vol_gt) / (vol_pred + vol_gt)

#############################
# 2) Main evaluation function
#############################
def evaluate_segmentation(pred_path, gt_path, labels=[1, 2, 3]):
    """
    Given file paths to predicted segmentation (pred_path) and ground-truth (gt_path),
    compute the Dice, Hausdorff distance, and Volumetric difference for each label.
    """
    # Load predicted and ground-truth images
    pred_nib = nib.load(pred_path)
    gt_nib = nib.load(gt_path)

    # Convert to numpy arrays
    pred_data = pred_nib.get_fdata()
    gt_data = gt_nib.get_fdata()

    if pred_data.ndim == 4 and pred_data.shape[-1] == 1:
      pred_data = np.squeeze(pred_data, axis=-1)

    if gt_data.ndim == 4 and gt_data.shape[-1] == 1:
      gt_data = np.squeeze(gt_data, axis=-1)

    # Make sure they're the same shape
    if pred_data.shape != gt_data.shape:
        raise ValueError(f"Predicted image and GT image have different shapes: "
                         f"{pred_data.shape} vs. {gt_data.shape}")

    # Optionally cast to integer type if your labels are integer-coded
    pred_data = pred_data.astype(np.int16)
    gt_data = gt_data.astype(np.int16)

    # Metrics storage
    dice_scores = []
    hd_scores = []
    vd_scores = []

    print(f"\nEvaluating segmentation:\n  Prediction: {pred_path}\n  Ground Truth: {gt_path}")
    for lbl in labels:
        dice = compute_dice_coefficient(pred_data, gt_data, label=lbl)
        hd = compute_hausdorff_distance(pred_data, gt_data, label=lbl)
        vd = compute_volumetric_difference(pred_data, gt_data, label=lbl)

        dice_scores.append(dice)
        hd_scores.append(hd)
        vd_scores.append(vd)

        print(f"\nLabel {lbl}:")
        print(f"  Dice Coefficient       = {dice if not np.isnan(dice) else 'NaN'}")
        print(f"  Hausdorff Distance     = {hd if not np.isnan(hd) else 'NaN'}")
        print(f"  Volumetric Difference  = {vd if not np.isnan(vd) else 'NaN'}")

    # Compute mean metrics (ignoring NaNs)
    mean_dice = np.nanmean(dice_scores)
    mean_hd = np.nanmean(hd_scores)
    mean_vd = np.nanmean(vd_scores)

    print("\nOverall mean metrics (labels = {}):".format(labels))
    print(f"  Mean Dice               = {mean_dice:.4f}")
    print(f"  Mean Hausdorff Distance = {mean_hd:.4f}")
    print(f"  Mean Volumetric Diff    = {mean_vd:.4f}")

    return dice_scores, hd_scores, vd_scores

##################################
# 3) Example usage (replace paths)
##################################
if __name__ == "__main__":
    # Example file paths; replace with your actual paths
    prediction_path = "/content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_17.nii.gz"
    ground_truth_path = "/content/drive/MyDrive/Dataset_Final/Validation_Set/IBSR_17/IBSR_17_seg.nii.gz"

    # If you want to include background (label=0), set labels=[0,1,2,3]
    evaluate_segmentation(prediction_path, ground_truth_path, labels=[1,2,3])



Evaluating segmentation:
  Prediction: /content/drive/MyDrive/Dataset_Final/segmented_images/validation/Segmented_IBSR_17.nii.gz
  Ground Truth: /content/drive/MyDrive/Dataset_Final/Validation_Set/IBSR_17/IBSR_17_seg.nii.gz

Label 1:
  Dice Coefficient       = 0.9274370208105148
  Hausdorff Distance     = 9.1104335791443
  Volumetric Difference  = 0.028008136441871383

Label 2:
  Dice Coefficient       = 0.9434103565319676
  Hausdorff Distance     = 11.045361017187261
  Volumetric Difference  = 0.001779741418114582

Label 3:
  Dice Coefficient       = 0.9248003255615006
  Hausdorff Distance     = 9.16515138991168
  Volumetric Difference  = 0.016911475933500564

Overall mean metrics (labels = [1, 2, 3]):
  Mean Dice               = 0.9319
  Mean Hausdorff Distance = 9.7736
  Mean Volumetric Diff    = 0.0156


# Test Segmentation using Ensemble

In [ ]:
# Install required libraries
!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

# Import required libraries
import os
import numpy as np
import torch
import torch.nn.functional as F
import nibabel as nib
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd, EnsureTyped
)
from monai.networks.nets import SwinUNETR, UNet
from monai.utils import set_determinism

# Set determinism
set_determinism(seed=0)

# ----------------------------
# 1) Data setup
# ----------------------------
test_data_dir = '/content/drive/MyDrive/Dataset_Final/Test_Set/'  # Path to the test dataset
test_files = [
    {'image': os.path.join(test_data_dir, 'IBSR_02/IBSR_02.nii.gz'), 'name': 'IBSR_02'},
    {'image': os.path.join(test_data_dir, 'IBSR_10/IBSR_10.nii.gz'), 'name': 'IBSR_10'},
    {'image': os.path.join(test_data_dir, 'IBSR_15/IBSR_15.nii.gz'), 'name': 'IBSR_15'},
]

test_transforms = Compose([
    LoadImaged(keys=['image']),
    EnsureChannelFirstd(keys=['image']),
    ScaleIntensityd(keys=['image']),
    EnsureTyped(keys=['image']),
])

# ----------------------------
# 2) DataLoader for test data
# ----------------------------
from monai.data import Dataset, DataLoader

num_workers = 4
test_ds = Dataset(data=test_files, transform=test_transforms)
test_loader = DataLoader(test_ds, batch_size=1, shuffle=False, num_workers=num_workers)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# 3) Define the ensemble models
# ----------------------------
model_swin = SwinUNETR(
    in_channels=1,
    out_channels=4,  # number of classes
    img_size=[96, 96, 96],
    feature_size=48,
    use_checkpoint=False,
).to(device)

model_unet = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=4,  # number of classes
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# ----------------------------
# 4) Load the saved ensemble checkpoint
# ----------------------------
model_save_path = '/content/drive/MyDrive/best_metric_model_ensemble.pth'
checkpoint = torch.load(model_save_path, map_location=device)

model_swin.load_state_dict(checkpoint["model_swin"])
model_unet.load_state_dict(checkpoint["model_unet"])

model_swin.eval()
model_unet.eval()

# ----------------------------
# 5) Segment and save test data
# ----------------------------
segmented_dir = '/content/drive/MyDrive/Dataset_Final/segmented_images_test/'
os.makedirs(segmented_dir, exist_ok=True)

# Define the voxel spacing and origin
target_spacing = [1, 1.5, 1]  # Spacing: x, y, z
target_origin = [255, 0, 0]  # Origin: x, y, z

with torch.no_grad():
    for test_data in test_loader:
        test_images = test_data["image"].to(device)  # shape: (B,1,D,H,W)
        test_name = test_data['name'][0]  # Extract file name from the data dict

        # Ensemble inference
        test_swin = model_swin(test_images)
        test_unet = model_unet(test_images)
        test_ensemble = 0.5 * test_swin + 0.5 * test_unet

        # Argmax of ensemble
        test_ensemble_argmax = torch.argmax(test_ensemble, dim=1)  # (B, D,H,W)

        pred_np = test_ensemble_argmax.cpu().numpy()  # shape: (B, D,H,W)

        for b in range(pred_np.shape[0]):
            pred = pred_np[b]  # shape: (D,H,W)

            # Create affine matrix for desired spacing and origin
            affine = np.array([
                [target_spacing[0], 0, 0, target_origin[0]],
                [0, target_spacing[1], 0, target_origin[1]],
                [0, 0, target_spacing[2], target_origin[2]],
                [0, 0, 0, 1],
            ])

            # Save the predicted segmentation as a NIfTI file
            output_path = os.path.join(segmented_dir, f"Segmented_{test_name}.nii.gz")
            pred_img = nib.Nifti1Image(pred.astype(np.int16), affine)
            nib.save(pred_img, output_path)
            print(f"Saved segmentation to {output_path}")


<ipython-input-11-0bf9c60c3de7>:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_save_path, map_location=device)


Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images_test/Segmented_IBSR_02.nii.gz
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images_test/Segmented_IBSR_10.nii.gz
Saved segmentation to /content/drive/MyDrive/Dataset_Final/segmented_images_test/Segmented_IBSR_15.nii.gz


# UNet Implementation

In [ ]:
# Install MONAI and other dependencies
!pip install monai==0.9.1
!pip install nibabel
!pip install SimpleITK

# Import necessary libraries
import os
import glob
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import nibabel as nib
import SimpleITK as sitk
import monai
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    RandSpatialCropd, RandFlipd, RandRotate90d,
    EnsureTyped
)
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.utils import set_determinism

from google.colab import drive
drive.mount("/content/drive")

# Set determinism for reproducibility
set_determinism(seed=0)

# Define the data directory
data_dir = '/content/drive/MyDrive/Dataset_Final/'  # Update this path if different

# Function to load data dictionaries
def get_data_dicts(data_dir, set_name):
    data_pattern = os.path.join(data_dir, set_name, 'IBSR_*')
    subject_dirs = glob.glob(data_pattern)
    data_dicts = []

    for subject_dir in subject_dirs:
        subject_name = os.path.basename(subject_dir)
        img_file = os.path.join(subject_dir, '{}.nii.gz'.format(subject_name))
        seg_file = os.path.join(subject_dir, '{}_seg.nii.gz'.format(subject_name))
        data_dicts.append({'image': img_file, 'label': seg_file})

    return data_dicts

# Get training and validation data dictionaries
train_files = get_data_dicts(data_dir, 'Training_Set')
val_files = get_data_dicts(data_dir, 'Validation_Set')

# Define training transformations
train_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    RandSpatialCropd(keys=['image', 'label'], roi_size=(96, 96, 96), random_size=False),
    RandFlipd(keys=['image', 'label'], prob=0.5, spatial_axis=0),
    RandRotate90d(keys=['image', 'label'], prob=0.5, max_k=3),
    EnsureTyped(keys=['image', 'label']),
])

# Define validation transformations
val_transforms = Compose([
    LoadImaged(keys=['image', 'label']),
    EnsureChannelFirstd(keys=['image', 'label']),
    ScaleIntensityd(keys=['image']),
    EnsureTyped(keys=['image', 'label']),
])

# Use CacheDataset for faster data loading
num_workers = 4  # Adjust based on your system's capabilities

# Create training and validation datasets and dataloaders
train_ds = CacheDataset(
    data=train_files,
    transform=train_transforms,
    cache_rate=1.0,  # Cache all data
    num_workers=num_workers
)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=num_workers)

val_ds = CacheDataset(
    data=val_files,
    transform=val_transforms,
    cache_rate=1.0,  # Cache all data
    num_workers=num_workers
)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=num_workers)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model (using MONAI's UNet)
model = UNet(
    dimensions=3,
    in_channels=1,
    out_channels=4,  # Number of classes
    channels=(32, 64, 128, 256, 512),
    strides=(2, 2, 2, 2),
    num_res_units=2,
    norm=Norm.BATCH,
).to(device)

# Optionally, load pre-trained weights if available
# model.load_state_dict(torch.load('/path/to/pretrained/model.pth'))

# Define loss function and optimizer
loss_function = DiceLoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Define metric for evaluation without to_onehot_y and softmax
dice_metric = DiceMetric(
    include_background=True,
    reduction="mean",
    get_not_nans=False,
)

# Training loop
max_epochs = 1000
val_interval = 2  # Do validation every 2 epochs
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []

for epoch in range(max_epochs):
    print('-' * 10)
    print('Epoch {}/{}'.format(epoch + 1, max_epochs))
    model.train()
    epoch_loss = 0
    step = 0
    for batch_data in train_loader:
        step += 1
        inputs, labels = batch_data['image'].to(device), batch_data['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print('{}/{}, train_loss: {:.4f}'.format(step, len(train_loader), loss.item()))
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print('Epoch {} average loss: {:.4f}'.format(epoch + 1, epoch_loss))

    if (epoch + 1) % val_interval == 0:
        model.eval()
        with torch.no_grad():
            metric_sum = 0.0
            metric_count = 0
            for val_data in val_loader:
                val_images = val_data['image'].to(device)
                val_labels = val_data['label'].to(device)
                val_outputs = model(val_images)

                # Apply softmax to model outputs
                val_outputs_softmax = F.softmax(val_outputs, dim=1)

                # Convert labels to one-hot encoding
                # Ensure labels are of type LongTensor
                val_labels_onehot = F.one_hot(val_labels.squeeze(1).long(), num_classes=4)  # Shape: (1, H, W, D, C)
                val_labels_onehot = val_labels_onehot.permute(0, 4, 1, 2, 3).float()  # Shape: (1, C, H, W, D)

                # Update DiceMetric
                dice_metric(y_pred=val_outputs_softmax, y=val_labels_onehot)

                metric_count += 1

            # Compute the mean dice score
            metric = dice_metric.aggregate().item()
            dice_metric.reset()
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), 'best_metric_model.pth')
                print('Saved new best metric model')
            print('Current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}'.format(
                epoch + 1, metric, best_metric, best_metric_epoch))

print('Training completed, best_metric: {:.4f} at epoch: {}'.format(best_metric, best_metric_epoch))

# Load the best model for evaluation
model.load_state_dict(torch.load('best_metric_model.pth'))

# Evaluation functions
def compute_dice_coefficient(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    intersection = np.logical_and(in1, in2).sum()
    volumes = in1.sum() + in2.sum()
    if volumes == 0:
        return np.NaN
    else:
        return 2. * intersection / volumes

def compute_hausdorff_distance(in1, in2, label=1):
    in1 = (in1 == label).astype(np.uint8)
    in2 = (in2 == label).astype(np.uint8)
    in1_sitk = sitk.GetImageFromArray(in1)
    in2_sitk = sitk.GetImageFromArray(in2)
    hausdorff_distance_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_distance_filter.Execute(in1_sitk, in2_sitk)
    return hausdorff_distance_filter.GetHausdorffDistance()

def compute_volumetric_difference(in1, in2, label=1):
    in1 = in1 == label
    in2 = in2 == label
    vol1 = in1.sum()
    vol2 = in2.sum()
    if vol1 + vol2 == 0:
        return np.NaN
    else:
        return abs(vol1 - vol2) / (vol1 + vol2)

# Evaluate on validation data
model.eval()
dice_scores = {i: [] for i in range(4)}  # Assuming 4 classes
hausdorff_distances = {i: [] for i in range(4)}
volumetric_differences = {i: [] for i in range(4)}

with torch.no_grad():
    for val_data in val_loader:
        val_images = val_data['image'].to(device)
        val_labels = val_data['label'].cpu().numpy()
        outputs = model(val_images)
        outputs = torch.argmax(F.softmax(outputs, dim=1), dim=1).cpu().numpy()  # Shape: (1, H, W, D)

        for label in range(4):  # For each class
            dice = compute_dice_coefficient(outputs[0], val_labels[0][0], label=label)
            hd = compute_hausdorff_distance(outputs[0], val_labels[0][0], label=label)
            vd = compute_volumetric_difference(outputs[0], val_labels[0][0], label=label)
            if not np.isnan(dice):
                dice_scores[label].append(dice)
            if not np.isnan(hd):
                hausdorff_distances[label].append(hd)
            if not np.isnan(vd):
                volumetric_differences[label].append(vd)

# Compute average metrics per class
for label in range(4):
    avg_dice = np.mean(dice_scores[label]) if dice_scores[label] else np.NaN
    avg_hd = np.mean(hausdorff_distances[label]) if hausdorff_distances[label] else np.NaN
    avg_vd = np.mean(volumetric_differences[label]) if volumetric_differences[label] else np.NaN
    print(f"Class {label}:")
    print(f"\tDice Coefficient = {avg_dice:.4f}")
    print(f"\tHausdorff Distance = {avg_hd:.4f}")
    print(f"\tVolumetric Difference = {avg_vd:.4f}")


  Using cached monai-0.9.1-202207251608-py3-none-any.whl.metadata (7.5 kB)
Using cached monai-0.9.1-202207251608-py3-none-any.whl (990 kB)
  Attempting uninstall: monai
    Found existing installation: monai 1.4.0
    Uninstalling monai-1.4.0:
      Successfully uninstalled monai-1.4.0
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loading dataset:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [0.8370536 1.5       0.8370536 0.       ] to [  0.8370536   1.5         0.8370536 213.4510119]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [1.  1.5 1.  0. ] to [  1.           1.5          1.         255.00196078]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
/usr/local/lib/python3.10/dist-packages/monai/data/utils.py:766: UserWarning: Modifying image pixdim from [0.9375 1.5    0.9375 0.    ] to [  0.9375       1.5          0.9375     239.06459149]
  warnings.warn(f"Modifying image pixdim from {pixdim} to {norm}")
Loading dataset: 100%|██████████| 5/5 [00:00<00:00,  5.78it/s]


----------
Epoch 1/1000


/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  ret = func(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_tensor.py:1512: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages

1/10, train_loss: 0.7879
2/10, train_loss: 0.8550
3/10, train_loss: 0.7899
4/10, train_loss: 0.8124
5/10, train_loss: 0.8008
6/10, train_loss: 0.7958
7/10, train_loss: 0.8105
8/10, train_loss: 0.7937
9/10, train_loss: 0.8449
10/10, train_loss: 0.7273
Epoch 1 average loss: 0.8018
----------
Epoch 2/1000
1/10, train_loss: 0.7309
2/10, train_loss: 0.7582
3/10, train_loss: 0.8304
4/10, train_loss: 0.7275
5/10, train_loss: 0.7030
6/10, train_loss: 0.7007
7/10, train_loss: 0.7619
8/10, train_loss: 0.6929
9/10, train_loss: 0.6794
10/10, train_loss: 0.7472
Epoch 2 average loss: 0.7332


/usr/local/lib/python3.10/dist-packages/monai/metrics/utils.py:219: UserWarning: y_pred should be a binarized tensor.
  warnings.warn(f"{name} should be a binarized tensor.")


Streaming output truncated to the last 5000 lines.
Current epoch: 630 current mean dice: 0.8353 best mean dice: 0.8711 at epoch 610
----------
Epoch 631/1000
1/10, train_loss: 0.0803
2/10, train_loss: 0.0664
3/10, train_loss: 0.0705
4/10, train_loss: 0.1633
5/10, train_loss: 0.0960
6/10, train_loss: 0.0605
7/10, train_loss: 0.1129
8/10, train_loss: 0.0833
9/10, train_loss: 0.0761
10/10, train_loss: 0.0779
Epoch 631 average loss: 0.0887
----------
Epoch 632/1000
1/10, train_loss: 0.1041
2/10, train_loss: 0.0747
3/10, train_loss: 0.1139
4/10, train_loss: 0.1252
5/10, train_loss: 0.0900
6/10, train_loss: 0.0865
7/10, train_loss: 0.0767
8/10, train_loss: 0.0746
9/10, train_loss: 0.1801
10/10, train_loss: 0.0628
Epoch 632 average loss: 0.0989
Current epoch: 632 current mean dice: 0.8677 best mean dice: 0.8711 at epoch 610
----------
Epoch 633/1000
1/10, train_loss: 0.1693
2/10, train_loss: 0.1838
3/10, train_loss: 0.2310
4/10, train_loss: 0.1108
5/10, train_loss: 0.0603
6/10, train_loss: 0.

<ipython-input-1-229b0c493f6e>:190: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_metric_model.pth'))


Class 0:
	Dice Coefficient = 0.9963
	Hausdorff Distance = 23.6909
	Volumetric Difference = 0.0018
Class 1:
	Dice Coefficient = 0.8372
	Hausdorff Distance = 21.4847
	Volumetric Difference = 0.0781
Class 2:
	Dice Coefficient = 0.8845
	Hausdorff Distance = 10.7371
	Volumetric Difference = 0.0325
Class 3:
	Dice Coefficient = 0.8522
	Hausdorff Distance = 11.6964
	Volumetric Difference = 0.1046
